In [1]:
# !pip install chromedriver-binary==119.0.6045.105

In [2]:
from datetime import datetime, timedelta

# 開始日と終了日を設定
start_date = datetime(2013, 11, 1)
end_date = datetime(2023, 11, 9)

# 現在の日付を開始日に設定
current_date = start_date

# 日付リストを初期化
date_list = []

# 開始日から終了日までループ
while current_date <= end_date:
    # 日付を指定された形式の文字列に変換し、整数にキャストしてリストに追加
    date_list.append(int(current_date.strftime('%Y%m%d')))
    # 次の日に更新
    current_date += timedelta(days=1)


In [3]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_binary
import time


# オプションを設定してヘッドレスモードを有効にする
chrome_options = Options()
chrome_options.add_argument("--headless")  # GUIをオフに
chrome_options.add_argument("--disable-gpu")  # GPUハードウェアアクセラレーションをオフに。Windows環境では必要な場合があります。
chrome_options.add_argument("--no-sandbox")  # サンドボックスプロセスを無効に。Linux環境の一部のセキュリティ制約を回避するために必要な場合があります。

# ヘッドレスモードでChromeを起動
driver = webdriver.Chrome(options=chrome_options)
# driver = webdriver.Chrome()

In [4]:
from tqdm import tqdm

data_list = []
error_list=[]
for date in tqdm(date_list):
    try:
        time.sleep(0.1)
        # スクレイピング対象のURL
        url = f'https://www.boatrace.jp/owpc/pc/race/index?hd={date}'

        # スクレイピング対象のURLにアクセス
        driver.get(url)

        table_x_path = '/html/body/main/div/div/div/div[2]/div[3]/table'
        table = driver.find_element(by=By.XPATH, value=table_x_path)

        tbody_elements = table.find_elements(by=By.TAG_NAME, value='tbody')

        for tbody in tbody_elements:
            d_list = []
            d_list.append(date)
            # ボートレース場
            place_name = tbody.find_element(By.TAG_NAME, 'tr').find_elements(By.TAG_NAME, 'td')[0].find_element(By.TAG_NAME, 'img').get_attribute('alt')
            d_list.append(place_name)
            # グレード
            grade = tbody.find_element(By.TAG_NAME, 'tr').find_elements(By.TAG_NAME, 'td')[2].get_attribute('class')
            d_list.append(grade)
            # 開催時間帯
            when = tbody.find_element(By.TAG_NAME, 'tr').find_elements(By.TAG_NAME, 'td')[3].get_attribute('class')
            d_list.append(when)
            # タイトル
            title = tbody.find_element(By.TAG_NAME, 'tr').find_elements(By.TAG_NAME, 'td')[4].find_element(By.TAG_NAME, 'a').text
            d_list.append(title)
            # タイトルURL
            title_url = tbody.find_element(By.TAG_NAME, 'tr').find_elements(By.TAG_NAME, 'td')[4].find_element(By.TAG_NAME, 'a').get_attribute('href')
            d_list.append(title_url)
            race_info_elements = tbody.find_element(By.TAG_NAME, 'tr').find_elements(By.TAG_NAME, 'td')[6].find_elements(By.TAG_NAME, 'li')
            # 出走表URL
            racelist_url = race_info_elements[0].find_element(By.TAG_NAME, 'a').get_attribute('href')
            d_list.append(racelist_url)
            # オッズURL
            odds_url = race_info_elements[1].find_element(By.TAG_NAME, 'a').get_attribute('href')
            d_list.append(odds_url)
            # 直前情報URL
            beforeinfo_url = race_info_elements[2].find_element(By.TAG_NAME, 'a').get_attribute('href')
            d_list.append(beforeinfo_url)
            # コンピュータ予想URL
            pcexpect_url = race_info_elements[3].find_element(By.TAG_NAME, 'a').get_attribute('href')
            d_list.append(pcexpect_url)
            # マイ予想URL
            myexpect_url = race_info_elements[4].find_element(By.TAG_NAME, 'a').get_attribute('href')
            d_list.append(myexpect_url)
            # 結果URL
            raceresult_url = race_info_elements[5].find_element(By.TAG_NAME, 'a').get_attribute('href')
            d_list.append(raceresult_url)
            # 結果一覧URL
            resultlist_url = race_info_elements[6].find_element(By.TAG_NAME, 'a').get_attribute('href')
            d_list.append(resultlist_url)
            # レース場データ
            stadium_url = tbody.find_element(By.TAG_NAME, 'tr').find_elements(By.TAG_NAME, 'td')[7].find_element(By.TAG_NAME, 'a').get_attribute('href')
            d_list.append(stadium_url)
            # リプレイデータ
            replay_url = tbody.find_element(By.TAG_NAME, 'tr').find_elements(By.TAG_NAME, 'td')[8].find_element(By.TAG_NAME, 'a').get_attribute('href')
            d_list.append(replay_url)

            data_list.append(d_list)
    except:
        print(f'error: {date}')
        error_list.append(date)
        continue

 20%|███▋              | 745/3661 [2:33:54<11:14:13, 13.87s/it]

error: 20151115


100%|███████████████████| 3661/3661 [12:50:33<00:00, 12.63s/it]


In [10]:
import pandas as pd


columns = [
    'date', 'place_name', 'grade', 'when', 'title', 'title_url', 'racelist_url', 'odds_url', 'beforeinfo_url',
    'pcexpect_url', 'myexpect_url', 'raceresult_url', 'resultlist_url', 'replay_url', 'stadium_url']
race_info_df = pd.DataFrame(data_list, columns=columns)
race_info_df

,date,place_name,grade,when,title,title_url,racelist_url,odds_url,beforeinfo_url,pcexpect_url,myexpect_url,raceresult_url,resultlist_url,replay_url,stadium_url
0,20131101,桐生,is-ippan,is-nighter,第４６回サンケイスポーツ杯,https://www.boatrace.jp/owpc/pc/race/raceindex...,https://www.boatrace.jp/owpc/pc/race/racelist?...,https://www.boatrace.jp/owpc/pc/race/odds3t?rn...,https://www.boatrace.jp/owpc/pc/race/beforeinf...,https://www.boatrace.jp/owpc/pc/race/pcexpect?...,https://www.boatrace.jp/owpc/pc/race/myexpect?...,https://www.boatrace.jp/owpc/pc/race/raceresul...,https://www.boatrace.jp/owpc/pc/race/resultlis...,https://www.boatrace.jp/owpc/pc/data/stadium?j...,javascript:MultiOpen('http://livebb.jlc.ne.jp/...
1,20131101,戸田,is-ippan,,第２９回日本モーターボート選手会会長賞,https://www.boatrace.jp/owpc/pc/race/raceindex...,https://www.boatrace.jp/owpc/pc/race/racelist?...,https://www.boatrace.jp/owpc/pc/race/odds3t?rn...,https://www.boatrace.jp/owpc/pc/race/beforeinf...,https://www.boatrace.jp/owpc/pc/race/pcexpect?...,https://www.boatrace.jp/owpc/pc/race/myexpect?...,https://www.boatrace.jp/owpc/pc/race/raceresul...,https://www.boatrace.jp/owpc/pc/race/resultlis...,https://www.boatrace.jp/owpc/pc/data/stadium?j...,javascript:MultiOpen('http://livebb.jlc.ne.jp/...
2,20131101,平和島,is-ippan,,第２７回関東日刊紙ボートレース記者クラブ杯,https://www.boatrace.jp/owpc/pc/race/raceindex...,https://www.boatrace.jp/owpc/pc/race/racelist?...,https://www.boatrace.jp/owpc/pc/race/odds3t?rn...,https://www.boatrace.jp/owpc/pc/race/beforeinf...,https://www.boatrace.jp/owpc/pc/race/pcexpect?...,https://www.boatrace.jp/owpc/pc/race/myexpect?...,https://www.boatrace.jp/owpc/pc/race/raceresul...,https://www.boatrace.jp/owpc/pc/race/resultlis...,https://www.boatrace.jp/owpc/pc/data/stadium?j...,javascript:MultiOpen('https://www.heiwajima.gr...
3,20131101,常滑,is-G3b,,ＧⅢ新鋭リーグ第５戦若獅子杯,https://www.boatrace.jp/owpc/pc/race/raceindex...,https://www.boatrace.jp/owpc/pc/race/racelist?...,https://www.boatrace.jp/owpc/pc/race/odds3t?rn...,https://www.boatrace.jp/owpc/pc/race/beforeinf...,https://www.boatrace.jp/owpc/pc/race/pcexpect?...,https://www.boatrace.jp/owpc/pc/race/myexpect?...,https://www.boatrace.jp/owpc/pc/race/raceresul...,https://www.boatrace.jp/owpc/pc/race/resultlis...,https://www.boatrace.jp/owpc/pc/data/stadium?j...,javascript:MultiOpen('http://livebb.jlc.ne.jp/...
4,20131101,三国,is-ippan,,かんなづき第２戦,https://www.boatrace.jp/owpc/pc/race/raceindex...,https://www.boatrace.jp/owpc/pc/race/racelist?...,https://www.boatrace.jp/owpc/pc/race/odds3t?rn...,https://www.boatrace.jp/owpc/pc/race/beforeinf...,https://www.boatrace.jp/owpc/pc/race/pcexpect?...,https://www.boatrace.jp/owpc/pc/race/myexpect?...,https://www.boatrace.jp/owpc/pc/race/raceresul...,https://www.boatrace.jp/owpc/pc/race/resultlis...,https://www.boatrace.jp/owpc/pc/data/stadium?j...,javascript:MultiOpen('http://livebb.jlc.ne.jp/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46029,20231109,丸亀,is-ippan,is-nighter,にっぽん未来プロジェクト競走ｉｎまるがめ,https://www.boatrace.jp/owpc/pc/race/raceindex...,https://www.boatrace.jp/owpc/pc/race/racelist?...,https://www.boatrace.jp/owpc/pc/race/odds3t?rn...,https://www.boatrace.jp/owpc/pc/race/beforeinf...,https://www.boatrace.jp/owpc/pc/race/pcexpect?...,https://www.boatrace.jp/owpc/pc/race/myexpect?...,https://www.boatrace.jp/owpc/pc/race/raceresul...,https://www.boatrace.jp/owpc/pc/race/resultlis...,https://www.boatrace.jp/owpc/pc/data/stadium?j...,javascript:MultiOpen('http://livebb.jlc.ne.jp/...
46030,20231109,宮島,is-ippan,,第１３回やまだ屋「桐葉菓」杯,https://www.boatrace.jp/owpc/pc/race/raceindex...,https://www.boatrace.jp/owpc/pc/race/racelist?...,https://www.boatrace.jp/owpc/pc/race/odds3t?rn...,https://www.boatrace.jp/owpc/pc/race/beforeinf...,https://www.boatrace.jp/owpc/pc/race/pcexpect?...,https://www.boatrace.jp/owpc/pc/race/myexpect?...,https://www.boatrace.jp/owpc/pc/race/raceresul...,https://www.boatrace.jp/owpc/pc/race/resultlis...,https://www.boatrace.jp/owpc/pc/data/stadium?j...,javascript:Mult

In [11]:
race_info_df.to_csv('race_info_df.csv')

In [12]:
error_list

[20151115]

In [94]:
date_list[:3]

[20131101, 20131102, 20131103]